In [21]:
! pip install geopy

In [22]:
import geopandas as gpd
from geopy.geocoders import Nominatim
import pandas as pd
from shapely.geometry import Point

### DO NOT REMOVE BELOW CODE BLOCK AS IT CONTAINS
### INTEGRAL INITIZALIZATION THAT MUST BE MOVED BEFORE DELETING

In [23]:
#DO NOT REMOVE DO NOT REMOVE THIS IS INITIZALIZING THE GEO LOCATOR
# REMOVING THIS WILL BREAK AND KILL US ALL
# Initialize geolocator
geolocator = Nominatim(user_agent="myexer")
# Coordinates (latitude, longitude)
coords = [(40.748817, -73.985428), (51.5074, -0.1278), (48.8566, 2.3522)]  # Example coordinates
# Function to geocode coordinates
def geocode_coordinates(lat, lon):
    location = geolocator.reverse((lat, lon), language='en')
    return location.address if location else None
# Create a list of addresses from the coordinates
addresses = [geocode_coordinates(lat, lon) for lat, lon in coords]
# Create a DataFrame
df = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Address'] = addresses
# Convert DataFrame to GeoDataFrame
geometry = [Point(lon, lat) for lat, lon in coords]
gdf = gpd.GeoDataFrame(df, geometry=geometry)
# Display the GeoDataFrame
print(gdf)

    Latitude  Longitude                                            Address  \
0  40.748817 -73.985428  Naturalizer, 7, West 34th Street, Midtown Sout...   
1  51.507400  -0.127800  Mileage Central Point of London Plaque, Charin...   
2  48.856600   2.352200  Hôtel de Ville, Place de l'Hôtel de Ville, Qua...   

                     geometry  
0  POINT (-73.98543 40.74882)  
1   POINT (-0.12780 51.50740)  
2    POINT (2.35220 48.85660)  


In [24]:
def my_geocode_coordinates(lat, lon):
    location = geolocator.reverse((lat, lon), language='en')
    return location if location else None

In [25]:
test = my_geocode_coordinates(40.837048, -73.865433)

In [26]:
test.address

'1475, Thieriot Avenue, The Bronx, Bronx County, New York, 10460, United States'

In [27]:
test.raw

{'place_id': 332816141,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 281860017,
 'lat': '40.8372181',
 'lon': '-73.86570288486502',
 'class': 'building',
 'type': 'yes',
 'place_rank': 30,
 'importance': 9.175936522464359e-05,
 'addresstype': 'building',
 'name': '',
 'display_name': '1475, Thieriot Avenue, The Bronx, Bronx County, New York, 10460, United States',
 'address': {'house_number': '1475',
  'road': 'Thieriot Avenue',
  'suburb': 'The Bronx',
  'city_district': 'Bronx County',
  'city': 'New York',
  'state': 'New York',
  'ISO3166-2-lvl4': 'US-NY',
  'postcode': '10460',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['40.8368253', '40.8375532', '-73.8658881', '-73.8654622']}

In [28]:
#create  with uuid, lat, long that geocodes each row
#Creates new all new geographic data into new dataset
#Join in to new dataset  with cleaned dataset
#For row in lat long columns grab geographic attributes into 


In [29]:
# records = []

# for row in my_geo_data:
#     trio = {}
#     trip['uuid'] = row['uuid']
#     trip['city'] = row['city']
#     record['state'] = row['state']

#     records.append(record)


# new_dataset = pd.DataFrame(records)

# new_dataset.to_csv("")



In [30]:
import pandas as pd
import os
print(os.getcwd())


df = pd.read_csv('../data/processed/uber_data_cleaned.csv')
df.rename(columns={'Unnamed: 0': 'UUID'}, inplace=True)
print(df.head())

/Users/mattrodosky/Desktop/repos/Project-2/notebooks
       UUID                            key  fare_amount  \
0  24238194    2015-05-07 19:52:06.0000003          7.5   
1  27835199    2009-07-17 20:04:56.0000002          7.7   
2  44984355   2009-08-24 21:45:00.00000061         12.9   
3  25894730    2009-06-26 08:22:21.0000001          5.3   
4  17610152  2014-08-28 17:47:00.000000188         16.0   

             pickup_datetime  pickup_longitude  pickup_latitude  \
0  2015-05-07 19:52:06+00:00        -73.999817        40.738354   
1  2009-07-17 20:04:56+00:00        -73.994355        40.728225   
2  2009-08-24 21:45:00+00:00        -74.005043        40.740770   
3  2009-06-26 08:22:21+00:00        -73.976124        40.790844   
4  2014-08-28 17:47:00+00:00        -73.925023        40.744085   

   dropoff_longitude  dropoff_latitude  passenger_count pickup_time  
0         -73.999512         40.723217                1    19:52:06  
1         -73.994710         40.750325           

In [39]:
def get_info(lat, lon):
    location = geolocator.reverse((lat, lon), exactly_one=True)
    if location:
        # Use the get method to safely access keys
        address = location.raw.get('address', {})
        return {
            'address_type': location.raw.get('type', None),
            'road': address.get('road', None),
            'suburb': address.get('suburb', None),
            'city': address.get('city', None),
            'state': address.get('state', None),
            'postcode': address.get('postcode', None),
            'country': address.get('country', None)
        }
    else:
        return None

In [40]:
#List to hold records to add to DF
records = {}

#Loop through rows
for index, row in df.iterrows():
    uuid = row['UUID']
    #Get pickup_address
    pickup_info = get_info(row['pickup_latitude'], row['pickup_longitude'])
    #Get dropoff_address
    dropoff_info = get_info(row['dropoff_latitude'], row['dropoff_longitude'])

    #Storing the results in the dictionary
    records[uuid] = {
        'pickup_info': pickup_info,
        'dropoff_info': dropoff_info
    }

#Print results
print(records)
    